# Add batch effects

Say we are interested in identifying genes that differentiate between disease vs normal states.  However our dataset includes samples from different tissues or time points and there are variations in gene expression that are due to these other conditions and do not have to do with disease state.  These non-relevant variations in the data are called *batch effects*.  

We want to model these batch effects.  To do this we will:
1. Partition our simulated data into n batches
2. For each partition we will randomly shift the expression data.  We randomly generate a binary vector of length=number of genes (*offset vector*).  This vector will serve as the direction that we will shift to.  Then we also have a random scalar that will tell us how big of a step to take in our random direction (*stretch factor*).  We shift our partitioned data by: batch effect partition = partitioned data + stretch factor * offset vector
3. Repeat this for each partition
4. Append all batch effect partitions together


In [1]:
%load_ext autoreload
%autoreload 2

import os
import ast
import pandas as pd
import numpy as np
import random
import glob
import umap
import pickle
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

from ggplot import *
from sklearn.decomposition import PCA
from numpy.random import seed
randomState = 123
seed(randomState)

In [2]:
# Load config file
config_file = "config_exp_1.txt"

d = {}
float_params = ["learning_rate", "kappa", "epsilon_std"]
str_params = ["analysis_name", "NN_architecture"]
lst_params = ["num_batches"]
with open(config_file) as f:
    for line in f:
        (name, val) = line.split()
        if name in float_params:
            d[name] = float(val)
        elif name in str_params:
            d[name] = str(val)
        elif name in lst_params:
            d[name] = ast.literal_eval(val)
        else:
            d[name] = int(val)

In [3]:
# Parameters
analysis_name = d["analysis_name"]
NN_architecture = d["NN_architecture"]
num_PCs = d["num_PCs"]
num_batches = d["num_batches"]

In [4]:
# Create directories
base_dir = os.path.abspath(os.path.join(os.getcwd(),"../.."))

new_dir = os.path.join(
    base_dir,
    "data",
    "batch_simulated")

analysis_dir = os.path.join(new_dir, analysis_name)

if os.path.exists(analysis_dir):
    print('directory already exists: {}'.format(analysis_dir))
else:
    print('creating new directory: {}'.format(analysis_dir))
os.makedirs(analysis_dir, exist_ok=True)

directory already exists: /home/alexandra/Documents/Repos/Batch_effects_simulation/data/batch_simulated/experiment_1


In [5]:
# Load arguments
simulated_data_file = os.path.join(
    base_dir,
    "data",
    "simulated",
    analysis_name,
    "simulated_data.txt.xz")

umap_model_file = umap_model_file = os.path.join(
    base_dir,
    "models",  
    NN_architecture,
    "umap_model.pkl")

In [6]:
# Read in UMAP model
infile = open(umap_model_file, 'rb')
umap_model = pickle.load(infile)
infile.close()

In [7]:
# Read in data
simulated_data = pd.read_table(
    simulated_data_file,
    header=0, 
    index_col=0,
    compression='xz',
    sep='\t')

simulated_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0.474494,0.696870,0.238944,0.267328,0.583896,0.163173,0.235312,0.575820,0.132366,0.400795,...,0.757839,0.279478,0.561519,0.491499,0.449728,0.677076,0.124337,0.541154,0.720271,0.433547
1,0.441185,0.550324,0.219221,0.266000,0.590521,0.201786,0.212634,0.536296,0.177473,0.313939,...,0.810853,0.320016,0.542352,0.554965,0.502701,0.587785,0.175613,0.481203,0.498466,0.438769
2,0.521074,0.543742,0.336719,0.252867,0.599334,0.291142,0.229665,0.567645,0.187659,0.430471,...,0.446711,0.463937,0.392560,0.507871,0.508629,0.474983,0.246932,0.588248,0.579252,0.413991
3,0.596203,0.551936,0.345861,0.302489,0.531909,0.258675,0.264847,0.422646,0.267940,0.474586,...,0.693343,0.364793,0.536226,0.499298,0.492214,0.415613,0.294030,0.536930,0.344359,0.429401
4,0.482794,0.633886,0.303745,0.312830,0.450920,0.218819,0.216378,0.451298,0.235343,0.466791,...,0.520269,0.356043,0.461527,0.473496,0.540051,0.382430,0.265336,0.431511,0.512582,0.530147
5,0.426684,0.701658,0.408193,0.342460,0.431087,0.437039,0.298775,0.508043,0.216968,0.542441,...,0.577298,0.251121,0.411203,0.367234,0.420593,0.295283,0.319015,0.530860,0.503682,0.435850
6,0.381678,0.589432,0.270064,0.342137,0.629668,0.250987,0.172229,0.401775,0.235665,0.338248,...,0.510384,0.377027,0.484328,0.492013,0.541812,0.344019,0.273659,0.545264,0.394876,0.499558
7,0.400522,0.636906,0.228519,0.464320,0.672855,0.236873,0.168115,0.330595,0.260448,0.323219,...,0.587457,0.363698,0.611703,0.467752,0.573952,0.355191,0.274361,0.565848,0.402649,0.593904
8,0.501997,0.601793,0.210271,0.388671,0.570547,0.236643,0.168893,0.323124,0.232122,0.416512,...,0.779672,0.276626,0.620640,0.533968,0.474002,0.257428,0.229648,0.610038,0.489887,0.498294
9,0.428152,0.623297,0.294565,0.350955,0.502915,0.277763,0.251814,0.494694,0.219805,0.450239,...,0.482579,0.388641,0.509143,0.414734,0.456965,0.407720,0.275378,0.559786,0.527550,0.514114


In [8]:
%%time
# Add batch effects
num_simulated_samples = simulated_data.shape[0]
num_genes = simulated_data.shape[1]
subset_genes_to_change = np.random.RandomState(randomState).choice([0, 1], size=(num_genes), p=[3./4, 1./4])
    
for i in num_batches:
    print('Creating simulated data with {} batches..'.format(i))
    
    batch_file = os.path.join(
            base_dir,
            "data",
            "batch_simulated",
            analysis_name,
            "Batch_"+str(i)+".txt.xz")
    
    num_samples_per_batch = int(num_simulated_samples/i)
    
    if i == 1:        
        simulated_data.to_csv(batch_file, sep='\t', compression='xz')
        
    else:  
        batch_data_df = pd.DataFrame()
        
        simulated_data_draw = simulated_data
        for j in range(i):
            stretch_factor = np.random.uniform(1.0,1.5)
            
            # Randomly select samples
            batch_df = simulated_data_draw.sample(n=num_samples_per_batch, frac=None, replace=False)
            batch_df.columns = batch_df.columns.astype(str)
            
            # Update df to remove selected samples
            sampled_ids = list(batch_df.index)
            simulated_data_draw = simulated_data_draw.drop(sampled_ids)

            # Add batch effect
            subset_genes_to_change_tile = pd.DataFrame(
                pd.np.tile(
                    subset_genes_to_change,
                    (num_samples_per_batch, 1)),
                index=batch_df.index,
                columns=simulated_data.columns)

            offset_vector = pd.DataFrame(subset_genes_to_change_tile*stretch_factor)
            offset_vector.columns = offset_vector.columns.astype(str)
            batch_df = batch_df + offset_vector
            
            #batch_df = batch_df*stretch_factor

            # if any exceed 1 then set to 1 since gene expression is normalized
            batch_df[batch_df>=1.0] = 1.0

            # Append batched together
            batch_data_df = batch_data_df.append(batch_df)

            # Select a new direction (i.e. a new subset of genes to change)
            np.random.shuffle(subset_genes_to_change)
        
        # Save
        batch_data_df.to_csv(batch_file, sep='\t', compression='xz')

Creating simulated data with 1 batches..
Creating simulated data with 2 batches..
Creating simulated data with 5 batches..
Creating simulated data with 10 batches..
Creating simulated data with 20 batches..
Creating simulated data with 50 batches..
Creating simulated data with 100 batches..
Creating simulated data with 500 batches..
Creating simulated data with 1000 batches..
Creating simulated data with 2000 batches..
Creating simulated data with 3000 batches..
Creating simulated data with 6000 batches..
CPU times: user 31min 49s, sys: 6min 14s, total: 38min 4s
Wall time: 38min 2s


## Plot batch data using UMAP

In [9]:
"""
# Plot generated data 

for i in num_batches:
    batch_data_file = os.path.join(
        base_dir,
        "data",
        "batch_simulated",
        analysis_name,
        "Batch_"+str(i)+".txt")
    
    batch_data = pd.read_table(
        batch_data_file,
        header=0,
        sep='\t',
        index_col=0)
    
    # UMAP embedding of decoded batch data
    batch_data_UMAPencoded = umap_model.transform(batch_data)
    batch_data_UMAPencoded_df = pd.DataFrame(data=batch_data_UMAPencoded,
                                             index=batch_data.index,
                                             columns=['1','2'])
    
        
    g = ggplot(aes(x='1',y='2'), data=batch_data_UMAPencoded_df) + \
                geom_point(alpha=0.5) + \
                scale_color_brewer(type='qual', palette='Set2') + \
                ggtitle("{} Batches".format(i))
    
    print(g)"""

'\n# Plot generated data \n\nfor i in num_batches:\n    batch_data_file = os.path.join(\n        base_dir,\n        "data",\n        "batch_simulated",\n        analysis_name,\n        "Batch_"+str(i)+".txt")\n    \n    batch_data = pd.read_table(\n        batch_data_file,\n        header=0,\n        sep=\'\t\',\n        index_col=0)\n    \n    # UMAP embedding of decoded batch data\n    batch_data_UMAPencoded = umap_model.transform(batch_data)\n    batch_data_UMAPencoded_df = pd.DataFrame(data=batch_data_UMAPencoded,\n                                             index=batch_data.index,\n                                             columns=[\'1\',\'2\'])\n    \n        \n    g = ggplot(aes(x=\'1\',y=\'2\'), data=batch_data_UMAPencoded_df) +                 geom_point(alpha=0.5) +                 scale_color_brewer(type=\'qual\', palette=\'Set2\') +                 ggtitle("{} Batches".format(i))\n    \n    print(g)'

## Plot batch data using PCA

In [10]:
"""
# Plot generated data 

for i in num_batches:
    batch_data_file = os.path.join(
        base_dir,
        "data",
        "batch_simulated",
        analysis_name,
        "Batch_"+str(i)+".txt")
    
    batch_data = pd.read_table(
        batch_data_file,
        header=0,
        sep='\t',
        index_col=0)
    
    # PCA projection    
    pca = PCA(n_components=num_PCs)
    batch_data_PCAencoded = pca.fit_transform(batch_data)
    
    # Encode data using PCA model    
    batch_data_PCAencoded_df = pd.DataFrame(batch_data_PCAencoded,
                                         index=batch_data.index
                                         )
    
    g = sns.pairplot(batch_data_PCAencoded_df)
    g.fig.suptitle("Batch {}".format(i))
       
    # Select pairwise PC's to plot
    pc1 = 0
    pc2 = 2
    
    # Encode data using PCA model    
    batch_data_PCAencoded_df = pd.DataFrame(batch_data_PCAencoded[:,[pc1,pc2]],
                                         index=batch_data.index,
                                         columns=['PC {}'.format(pc1), 'PC {}'.format(pc2)])
    
    g = ggplot(aes(x='PC {}'.format(pc1),y='PC {}'.format(pc2)), data=batch_data_PCAencoded_df)  + \
                geom_point(alpha=0.5) + \
                scale_color_brewer(type='qual', palette='Set2') + \
                ggtitle("{} Batches".format(i))
    print(g)

"""

'\n# Plot generated data \n\nfor i in num_batches:\n    batch_data_file = os.path.join(\n        base_dir,\n        "data",\n        "batch_simulated",\n        analysis_name,\n        "Batch_"+str(i)+".txt")\n    \n    batch_data = pd.read_table(\n        batch_data_file,\n        header=0,\n        sep=\'\t\',\n        index_col=0)\n    \n    # PCA projection    \n    pca = PCA(n_components=num_PCs)\n    batch_data_PCAencoded = pca.fit_transform(batch_data)\n    \n    # Encode data using PCA model    \n    batch_data_PCAencoded_df = pd.DataFrame(batch_data_PCAencoded,\n                                         index=batch_data.index\n                                         )\n    \n    g = sns.pairplot(batch_data_PCAencoded_df)\n    g.fig.suptitle("Batch {}".format(i))\n       \n    # Select pairwise PC\'s to plot\n    pc1 = 0\n    pc2 = 2\n    \n    # Encode data using PCA model    \n    batch_data_PCAencoded_df = pd.DataFrame(batch_data_PCAencoded[:,[pc1,pc2]],\n                   